## Load the required classes
These classes contain the code to encode and decode the files and induce errors in the DNA strands. An instance is created of each class and stored into a variable, so that they can be used.

In [ ]:
from encoder import Encoder
from decoder import Decoder
from errorSimulator import ErrorSimulator

encoder         = Encoder()
decoder         = Decoder()
errorSimulator  = ErrorSimulator()

## Encoder

The encoder is used to convert the binary data into DNA-strands...

In [ ]:
inFile = "./test_data/input.txt"
outFile = "./test_data/output.txt"

encoder.encode(inFile, outFile)

## Decoder

The decoder is used to convert the DNA-strands back into binary files...

In [ ]:
inFile = "./test_data/output.txt"
outFile = "./test_data/test.txt"

encoder.encode(inFile, outFile)

## Error simulator

The error simulator is used to simulate errors in DNA-strands comming from PCR/synthesis...

In [ ]:
inFile = "./test_data/output.txt"
outFile = "./test_data/test2.txt"

errorSimulator.induceErrors(inFile, outFile)

## Testing decoder

Testing the decoder for the same settings

In [ ]:
error_infile = "./test_data/output.txt"
error_outfile = "./test_data/test.txt"
decoder_infile = "./test_data/test.txt"
decoder_outfile = "./test_data/test2.txt"


errorSimulator.induceErrors(error_infile, error_outfile)
col_errors, row_errors, succes = decoder.decode(decoder_infile, decoder_outfile)
print("Succes: ", succes)



## Testing decoder

Testing the decoder for different reduncancies

In [ ]:
import numpy as np
import pandas as pd

encoder_infile = "./test_data/check.txt"
encoder_outfile = "./test_data/output.txt"
error_infile = "./test_data/output.txt"
error_outfile = "./test_data/test.txt"
decoder_infile = "./test_data/test.txt"
decoder_outfile = "./test_data/test2.txt"

# Init result matrix
result = np.empty([20, 20])

# Number of samples per reduncancy combination
itters = 100

#Initialize dataframe
df = pd.DataFrame(columns = ['row redundancy', 'column redundancy', 'column errors', 'row errors', 'succes %'])
data = pd.DataFrame()


i = 0
for row_redundancy in range(450, 550, 50):
    j = 0
    encoder.set_row_redundancy(row_redundancy)
    decoder.set_row_redundancy(row_redundancy)
    for column_redundancy in range(12, 16 ,2):
        
        encoder.set_column_redundancy(column_redundancy)
        decoder.set_column_redundancy(column_redundancy)
        encoder.encode(encoder_infile, encoder_outfile)

        succesfull_decodings = 0

        for itter in range(itters):
            errorSimulator.induceErrors(error_infile, error_outfile, insert_rate=  0.000462)
            col_errors, row_errors, succes = decoder.decode(decoder_infile, decoder_outfile)

            print(itter)
            print(column_redundancy, row_redundancy)
            print("Column errors")
            print(col_errors.count(0)/len(col_errors))
            print(col_errors)
            print("Row errors")
            print(row_errors)
            print(succes[-1])
            print(succes)

            succesfull_decodings += int(all(succes))

        result[i,j] = succesfull_decodings/itters

        #Organize data
        data = {
            'row redundancy' : row_redundancy, 
            'column redundancy' : column_redundancy, 
            'column errors':succes[-1], 
            'row errors': 0, 
            'succes %': result[i,j]
        }

        #Append data to dataframe and write it to an excel file
        df.loc[len(df)] = data
        df.to_excel("./results/results.xlsx")

        print(row_redundancy, column_redundancy, succesfull_decodings/itters)




        j += 1
    i += 1

print(result)



        

